# sEMG-HHT CNN Classifier for Movement Quality and Fatigue Classification

This notebook implements a Convolutional Neural Network (CNN) encoder with SVM classifier for classifying surface electromyography (sEMG) signals based on movement quality and fatigue levels.

## Architecture Overview
- **Input**: 256×256 matrix from HHT (Hilbert-Huang Transform) of sEMG signals
- **Encoder**: 3-layer CNN with Conv2D + InstanceNorm + LeakyReLU
- **Pooling**: Global Average Pooling
- **Classifier**: SVM for multi-class classification

## Requirements
- PyTorch
- scikit-learn
- NumPy
- Matplotlib

In [ ]:
# Install dependencies (uncomment if running on Kaggle or fresh environment)
# !pip install torch torchvision scikit-learn numpy matplotlib scipy PyEMD

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from typing import Tuple, Optional
import warnings

# Suppress specific warnings that are not critical for this demo
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')

# Set random seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. CNN Encoder Architecture

The encoder consists of 3 convolutional layers, each with:
- Conv2D (kernel=3, stride=2, padding=1)
- Instance Normalization
- LeakyReLU activation

This progressively reduces the spatial dimensions while extracting features.

In [ ]:
class ConvBlock(nn.Module):
    """Convolutional block with Conv2D, InstanceNorm, and LeakyReLU."""
    
    def __init__(self, in_channels: int, out_channels: int, 
                 kernel_size: int = 3, stride: int = 2, padding: int = 1,
                 leaky_slope: float = 0.2):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, 
                              kernel_size=kernel_size, 
                              stride=stride, 
                              padding=padding)
        self.instance_norm = nn.InstanceNorm2d(out_channels)
        self.activation = nn.LeakyReLU(negative_slope=leaky_slope)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        x = self.instance_norm(x)
        x = self.activation(x)
        return x


class sEMGHHTEncoder(nn.Module):
    """
    CNN Encoder for sEMG-HHT matrix classification.
    
    Architecture:
    - Input: 1×256×256 (single-channel HHT matrix)
    - 3 ConvBlocks with increasing channels
    - Global Average Pooling
    - Output: Feature vector for SVM classification
    """
    
    def __init__(self, in_channels: int = 1, 
                 base_channels: int = 64,
                 num_layers: int = 3,
                 leaky_slope: float = 0.2):
        super(sEMGHHTEncoder, self).__init__()
        
        self.in_channels = in_channels
        self.base_channels = base_channels
        self.num_layers = num_layers
        
        # Build convolutional layers
        layers = []
        current_channels = in_channels
        
        for i in range(num_layers):
            out_channels = base_channels * (2 ** i)
            layers.append(ConvBlock(
                in_channels=current_channels,
                out_channels=out_channels,
                kernel_size=3,
                stride=2,
                padding=1,
                leaky_slope=leaky_slope
            ))
            current_channels = out_channels
        
        self.encoder = nn.Sequential(*layers)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Calculate output feature dimension
        self.feature_dim = base_channels * (2 ** (num_layers - 1))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through the encoder.
        
        Args:
            x: Input tensor of shape (batch, channels, height, width)
        
        Returns:
            Feature tensor of shape (batch, feature_dim)
        """
        x = self.encoder(x)
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)  # Flatten to (batch, feature_dim)
        return x
    
    def get_feature_dim(self) -> int:
        """Return the output feature dimension."""
        return self.feature_dim

In [ ]:
# Verify the architecture
encoder = sEMGHHTEncoder(in_channels=1, base_channels=64, num_layers=3)
print("Encoder Architecture:")
print(encoder)
print(f"\nOutput feature dimension: {encoder.get_feature_dim()}")

# Test with sample input
sample_input = torch.randn(4, 1, 256, 256)  # batch=4, channels=1, height=256, width=256
sample_output = encoder(sample_input)
print(f"\nInput shape: {sample_input.shape}")
print(f"Output shape: {sample_output.shape}")

## 2. Complete Classification Pipeline

This class combines the CNN encoder with an SVM classifier for end-to-end classification.

In [ ]:
class sEMGHHTClassifier:
    """
    Complete classification pipeline combining CNN encoder and SVM classifier.
    
    The pipeline:
    1. Extracts features using CNN encoder
    2. Normalizes features using StandardScaler
    3. Classifies using SVM (supports multi-class)
    """
    
    def __init__(self, 
                 encoder: Optional[sEMGHHTEncoder] = None,
                 svm_kernel: str = 'rbf',
                 svm_C: float = 1.0,
                 svm_gamma: str = 'scale',
                 device: torch.device = torch.device('cpu')):
        """
        Initialize the classifier.
        
        Args:
            encoder: Pre-trained or new CNN encoder (creates default if None)
            svm_kernel: SVM kernel type ('rbf', 'linear', 'poly')
            svm_C: SVM regularization parameter
            svm_gamma: SVM gamma parameter
            device: Device to run the encoder on
        """
        self.device = device
        
        # Initialize encoder
        if encoder is None:
            self.encoder = sEMGHHTEncoder(
                in_channels=1, 
                base_channels=64, 
                num_layers=3
            )
        else:
            self.encoder = encoder
        
        self.encoder.to(self.device)
        
        # Initialize scaler and SVM
        self.scaler = StandardScaler()
        self.svm = SVC(
            kernel=svm_kernel,
            C=svm_C,
            gamma=svm_gamma,
            decision_function_shape='ovr',  # One-vs-Rest for multi-class
            probability=True  # Enable probability estimates
        )
        
        self._is_fitted = False
    
    def extract_features(self, X: np.ndarray, batch_size: int = 32) -> np.ndarray:
        """
        Extract features from HHT matrices using the CNN encoder.
        
        Args:
            X: Input array of shape (n_samples, height, width) or (n_samples, 1, height, width)
            batch_size: Batch size for processing
        
        Returns:
            Feature array of shape (n_samples, feature_dim)
        """
        self.encoder.eval()
        
        # Ensure correct shape
        if X.ndim == 3:
            X = X[:, np.newaxis, :, :]  # Add channel dimension
        
        features_list = []
        n_samples = X.shape[0]
        
        with torch.no_grad():
            for i in range(0, n_samples, batch_size):
                batch = torch.tensor(X[i:i+batch_size], dtype=torch.float32).to(self.device)
                batch_features = self.encoder(batch)
                features_list.append(batch_features.cpu().numpy())
        
        return np.vstack(features_list)
    
    def fit(self, X: np.ndarray, y: np.ndarray, batch_size: int = 32):
        """
        Fit the classifier (extract features and train SVM).
        
        Args:
            X: Training HHT matrices of shape (n_samples, height, width)
            y: Training labels of shape (n_samples,)
            batch_size: Batch size for feature extraction
        """
        print("Extracting features from training data...")
        features = self.extract_features(X, batch_size)
        
        print("Normalizing features...")
        features_scaled = self.scaler.fit_transform(features)
        
        print("Training SVM classifier...")
        self.svm.fit(features_scaled, y)
        
        self._is_fitted = True
        print("Training complete!")
    
    def predict(self, X: np.ndarray, batch_size: int = 32) -> np.ndarray:
        """
        Predict class labels for samples.
        
        Args:
            X: Input HHT matrices of shape (n_samples, height, width)
            batch_size: Batch size for feature extraction
        
        Returns:
            Predicted labels of shape (n_samples,)
        """
        if not self._is_fitted:
            raise RuntimeError("Classifier must be fitted before predicting")
        
        features = self.extract_features(X, batch_size)
        features_scaled = self.scaler.transform(features)
        return self.svm.predict(features_scaled)
    
    def predict_proba(self, X: np.ndarray, batch_size: int = 32) -> np.ndarray:
        """
        Predict class probabilities for samples.
        
        Args:
            X: Input HHT matrices of shape (n_samples, height, width)
            batch_size: Batch size for feature extraction
        
        Returns:
            Probability array of shape (n_samples, n_classes)
        """
        if not self._is_fitted:
            raise RuntimeError("Classifier must be fitted before predicting")
        
        features = self.extract_features(X, batch_size)
        features_scaled = self.scaler.transform(features)
        return self.svm.predict_proba(features_scaled)
    
    def evaluate(self, X: np.ndarray, y: np.ndarray, 
                 batch_size: int = 32) -> dict:
        """
        Evaluate the classifier on test data.
        
        Args:
            X: Test HHT matrices
            y: True labels
            batch_size: Batch size for feature extraction
        
        Returns:
            Dictionary containing accuracy, predictions, and classification report
        """
        y_pred = self.predict(X, batch_size)
        accuracy = accuracy_score(y, y_pred)
        
        return {
            'accuracy': accuracy,
            'predictions': y_pred,
            'classification_report': classification_report(y, y_pred),
            'confusion_matrix': confusion_matrix(y, y_pred)
        }

## 3. Data Generation and HHT Simulation

For demonstration purposes, we create synthetic HHT matrices representing different movement quality and fatigue levels.

In [ ]:
def generate_synthetic_hht_data(n_samples_per_class: int = 100,
                                 n_classes: int = 4,
                                 matrix_size: int = 256,
                                 random_state: int = 42) -> Tuple[np.ndarray, np.ndarray, list]:
    """
    Generate synthetic HHT matrices for demonstration.
    
    Creates HHT-like matrices with different frequency and amplitude patterns
    to simulate different movement quality and fatigue levels:
    - Class 0: High quality, Low fatigue (strong, clear patterns)
    - Class 1: High quality, High fatigue (strong patterns, more noise)
    - Class 2: Low quality, Low fatigue (weaker patterns, clear)
    - Class 3: Low quality, High fatigue (weak patterns, noisy)
    
    Args:
        n_samples_per_class: Number of samples per class
        n_classes: Number of classes
        matrix_size: Size of the HHT matrix (matrix_size × matrix_size)
        random_state: Random seed for reproducibility
    
    Returns:
        X: Array of HHT matrices (n_samples, matrix_size, matrix_size)
        y: Array of labels
        class_names: List of class descriptions
    """
    np.random.seed(random_state)
    
    class_names = [
        'High Quality, Low Fatigue',
        'High Quality, High Fatigue',
        'Low Quality, Low Fatigue',
        'Low Quality, High Fatigue'
    ]
    
    # Parameters for each class
    class_params = [
        {'amplitude': 1.0, 'noise': 0.1, 'freq_spread': 0.3},  # High Q, Low F
        {'amplitude': 1.0, 'noise': 0.4, 'freq_spread': 0.5},  # High Q, High F
        {'amplitude': 0.5, 'noise': 0.1, 'freq_spread': 0.6},  # Low Q, Low F
        {'amplitude': 0.5, 'noise': 0.4, 'freq_spread': 0.8},  # Low Q, High F
    ]
    
    X_list = []
    y_list = []
    
    # Time and frequency axes
    t = np.linspace(0, 1, matrix_size)
    f = np.linspace(0, 100, matrix_size)
    T, F = np.meshgrid(t, f)
    
    for class_idx in range(min(n_classes, len(class_params))):
        params = class_params[class_idx]
        
        for _ in range(n_samples_per_class):
            # Generate base HHT-like pattern
            center_freq = 30 + np.random.uniform(-10, 10)
            center_time = 0.5 + np.random.uniform(-0.2, 0.2)
            
            # Create Gaussian-like energy distribution in time-frequency plane
            hht_matrix = params['amplitude'] * np.exp(
                -((F - center_freq) ** 2) / (2 * (10 * params['freq_spread']) ** 2)
                -((T - center_time) ** 2) / (2 * (0.2) ** 2)
            )
            
            # Add secondary components
            for _ in range(np.random.randint(1, 4)):
                sec_freq = np.random.uniform(10, 80)
                sec_time = np.random.uniform(0.2, 0.8)
                sec_amp = params['amplitude'] * np.random.uniform(0.2, 0.5)
                
                hht_matrix += sec_amp * np.exp(
                    -((F - sec_freq) ** 2) / (2 * (8 * params['freq_spread']) ** 2)
                    -((T - sec_time) ** 2) / (2 * (0.15) ** 2)
                )
            
            # Add noise
            noise = params['noise'] * np.random.randn(matrix_size, matrix_size)
            hht_matrix += noise
            
            # Normalize to [0, 1] range
            hht_matrix = (hht_matrix - hht_matrix.min()) / (hht_matrix.max() - hht_matrix.min() + 1e-8)
            
            X_list.append(hht_matrix)
            y_list.append(class_idx)
    
    X = np.array(X_list, dtype=np.float32)
    y = np.array(y_list)
    
    # Shuffle data
    shuffle_idx = np.random.permutation(len(y))
    X = X[shuffle_idx]
    y = y[shuffle_idx]
    
    return X, y, class_names[:n_classes]

In [ ]:
# Generate synthetic data
print("Generating synthetic HHT data...")
X, y, class_names = generate_synthetic_hht_data(
    n_samples_per_class=100,
    n_classes=4,
    matrix_size=256,
    random_state=SEED
)

print(f"Data shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Class names: {class_names}")
print(f"Class distribution: {np.bincount(y)}")

In [ ]:
# Visualize sample HHT matrices from each class
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for i, class_idx in enumerate(range(4)):
    # Find a sample from this class
    sample_idx = np.where(y == class_idx)[0][0]
    
    im = axes[i].imshow(X[sample_idx], aspect='auto', cmap='hot', 
                        extent=[0, 1, 0, 100], origin='lower')
    axes[i].set_title(f'Class {class_idx}: {class_names[class_idx]}')
    axes[i].set_xlabel('Time (normalized)')
    axes[i].set_ylabel('Frequency (Hz)')
    plt.colorbar(im, ax=axes[i], label='Amplitude')

plt.tight_layout()
plt.suptitle('Sample HHT Matrices from Each Class', y=1.02)
plt.show()

## 4. Training and Evaluation

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
# Initialize and train the classifier
classifier = sEMGHHTClassifier(
    encoder=None,  # Use default encoder
    svm_kernel='rbf',
    svm_C=10.0,
    svm_gamma='scale',
    device=device
)

# Train
classifier.fit(X_train, y_train, batch_size=16)

In [ ]:
# Evaluate on test set
print("Evaluating on test set...")
results = classifier.evaluate(X_test, y_test, batch_size=16)

print(f"\nTest Accuracy: {results['accuracy']:.4f}")
print("\nClassification Report:")
print(results['classification_report'])

In [ ]:
# Plot confusion matrix
fig, ax = plt.subplots(figsize=(8, 6))
cm = results['confusion_matrix']
im = ax.imshow(cm, interpolation='nearest', cmap='Blues')
ax.figure.colorbar(im, ax=ax)

# Set labels
ax.set(xticks=np.arange(cm.shape[1]),
       yticks=np.arange(cm.shape[0]),
       xticklabels=[f'Class {i}' for i in range(4)],
       yticklabels=[f'Class {i}' for i in range(4)],
       title='Confusion Matrix',
       ylabel='True Label',
       xlabel='Predicted Label')

# Rotate tick labels
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# Add text annotations
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], 'd'),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.show()

## 5. End-to-End Training with Encoder Fine-tuning (Optional)

For better performance, you can fine-tune the encoder alongside training a neural network classifier.

In [ ]:
class sEMGHHTEndToEndClassifier(nn.Module):
    """
    End-to-end trainable classifier with CNN encoder and linear classification head.
    
    This version allows the encoder to be fine-tuned during training.
    """
    
    def __init__(self, 
                 n_classes: int = 4,
                 in_channels: int = 1,
                 base_channels: int = 64,
                 num_encoder_layers: int = 3,
                 dropout_rate: float = 0.5):
        super(sEMGHHTEndToEndClassifier, self).__init__()
        
        self.encoder = sEMGHHTEncoder(
            in_channels=in_channels,
            base_channels=base_channels,
            num_layers=num_encoder_layers
        )
        
        feature_dim = self.encoder.get_feature_dim()
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(feature_dim, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, n_classes)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits
    
    def get_features(self, x: torch.Tensor) -> torch.Tensor:
        """Extract features without classification."""
        return self.encoder(x)

In [ ]:
def train_end_to_end(model: nn.Module,
                     X_train: np.ndarray,
                     y_train: np.ndarray,
                     X_val: np.ndarray,
                     y_val: np.ndarray,
                     epochs: int = 50,
                     batch_size: int = 16,
                     learning_rate: float = 0.001,
                     device: torch.device = torch.device('cpu')) -> dict:
    """
    Train the end-to-end model.
    
    Args:
        model: The model to train
        X_train, y_train: Training data and labels
        X_val, y_val: Validation data and labels
        epochs: Number of training epochs
        batch_size: Batch size
        learning_rate: Learning rate
        device: Device to train on
    
    Returns:
        Dictionary containing training history
    """
    model = model.to(device)
    
    # Ensure correct shape
    if X_train.ndim == 3:
        X_train = X_train[:, np.newaxis, :, :]
        X_val = X_val[:, np.newaxis, :, :]
    
    # Create data loaders
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.long)
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True
    )
    
    val_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_val, dtype=torch.float32),
        torch.tensor(y_val, dtype=torch.long)
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False
    )
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5
    )
    
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    best_val_acc = 0.0
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * batch_X.size(0)
            _, predicted = outputs.max(1)
            train_total += batch_y.size(0)
            train_correct += predicted.eq(batch_y).sum().item()
        
        train_loss /= train_total
        train_acc = train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                
                val_loss += loss.item() * batch_X.size(0)
                _, predicted = outputs.max(1)
                val_total += batch_y.size(0)
                val_correct += predicted.eq(batch_y).sum().item()
        
        val_loss /= val_total
        val_acc = val_correct / val_total
        
        # Update learning rate
        scheduler.step(val_acc)
        
        # Save history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}: "
                  f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                  f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    print(f"\nBest Validation Accuracy: {best_val_acc:.4f}")
    return history

In [ ]:
# Train end-to-end model
e2e_model = sEMGHHTEndToEndClassifier(
    n_classes=4,
    in_channels=1,
    base_channels=64,
    num_encoder_layers=3,
    dropout_rate=0.5
)

print("Training end-to-end model...")
history = train_end_to_end(
    model=e2e_model,
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    epochs=50,
    batch_size=16,
    learning_rate=0.001,
    device=device
)

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
axes[0].plot(history['train_loss'], label='Train Loss')
axes[0].plot(history['val_loss'], label='Val Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True)

# Accuracy plot
axes[1].plot(history['train_acc'], label='Train Accuracy')
axes[1].plot(history['val_acc'], label='Val Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training and Validation Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 6. Model Saving and Loading

In [ ]:
import pickle

def save_svm_classifier(classifier: sEMGHHTClassifier, path: str):
    """Save the SVM-based classifier to disk."""
    # Save encoder
    torch.save(classifier.encoder.state_dict(), f"{path}_encoder.pt")
    
    # Save scaler and SVM
    with open(f"{path}_scaler.pkl", 'wb') as f:
        pickle.dump(classifier.scaler, f)
    
    with open(f"{path}_svm.pkl", 'wb') as f:
        pickle.dump(classifier.svm, f)
    
    print(f"Classifier saved to {path}_*.pt/pkl")

def load_svm_classifier(path: str, device: torch.device = torch.device('cpu')) -> sEMGHHTClassifier:
    """Load a saved SVM-based classifier."""
    classifier = sEMGHHTClassifier(device=device)
    
    # Load encoder
    classifier.encoder.load_state_dict(torch.load(f"{path}_encoder.pt", map_location=device))
    
    # Load scaler and SVM
    with open(f"{path}_scaler.pkl", 'rb') as f:
        classifier.scaler = pickle.load(f)
    
    with open(f"{path}_svm.pkl", 'rb') as f:
        classifier.svm = pickle.load(f)
    
    classifier._is_fitted = True
    print(f"Classifier loaded from {path}_*.pt/pkl")
    return classifier

def save_e2e_model(model: nn.Module, path: str):
    """Save the end-to-end model."""
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def load_e2e_model(path: str, n_classes: int = 4, 
                   device: torch.device = torch.device('cpu')) -> sEMGHHTEndToEndClassifier:
    """Load a saved end-to-end model."""
    model = sEMGHHTEndToEndClassifier(n_classes=n_classes)
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    model.eval()
    print(f"Model loaded from {path}")
    return model

In [ ]:
# Example: Save models (uncomment to use)
# save_svm_classifier(classifier, 'semg_hht_classifier')
# save_e2e_model(e2e_model, 'semg_hht_e2e_model.pt')

## 7. Real HHT Transformation (For Reference)

When you have real sEMG data, you can use the following functions to perform HHT transformation.

In [ ]:
def compute_hht_matrix(signal: np.ndarray, 
                       fs: float, 
                       matrix_size: int = 256,
                       max_imf: int = 10) -> np.ndarray:
    """
    Compute HHT (Hilbert-Huang Transform) matrix from a signal.
    
    This is a reference implementation. For real usage, you may need to install:
    pip install PyEMD scipy
    
    Args:
        signal: 1D input signal
        fs: Sampling frequency
        matrix_size: Output matrix size (matrix_size × matrix_size)
        max_imf: Maximum number of IMFs to extract
    
    Returns:
        HHT matrix of shape (matrix_size, matrix_size)
    """
    try:
        from PyEMD import EMD
        from scipy.signal import hilbert
    except ImportError:
        raise ImportError("Please install PyEMD and scipy: pip install PyEMD scipy")
    
    # Perform EMD
    emd = EMD()
    imfs = emd(signal, max_imf=max_imf)
    
    # Compute Hilbert transform for each IMF
    n_samples = len(signal)
    t = np.arange(n_samples) / fs
    
    # Initialize time-frequency matrix
    freq_bins = np.linspace(0, fs/2, matrix_size)
    time_bins = np.linspace(0, t[-1], matrix_size)
    hht_matrix = np.zeros((matrix_size, matrix_size))
    
    for imf in imfs:
        # Compute analytic signal
        analytic = hilbert(imf)
        amplitude = np.abs(analytic)
        phase = np.unwrap(np.angle(analytic))
        
        # Compute instantaneous frequency
        inst_freq = np.diff(phase) / (2 * np.pi) * fs
        inst_freq = np.concatenate([inst_freq, [inst_freq[-1]]])
        inst_freq = np.clip(inst_freq, 0, fs/2)
        
        # Map to time-frequency matrix
        for i, (ti, fi, ai) in enumerate(zip(t, inst_freq, amplitude)):
            t_idx = int(ti / t[-1] * (matrix_size - 1))
            f_idx = int(fi / (fs/2) * (matrix_size - 1))
            
            t_idx = np.clip(t_idx, 0, matrix_size - 1)
            f_idx = np.clip(f_idx, 0, matrix_size - 1)
            
            hht_matrix[f_idx, t_idx] += ai
    
    # Normalize
    if hht_matrix.max() > 0:
        hht_matrix = hht_matrix / hht_matrix.max()
    
    return hht_matrix.astype(np.float32)


# Example usage (uncomment when you have real data):
# signal = np.random.randn(1000)  # Replace with real sEMG signal
# fs = 1000  # Sampling frequency in Hz
# hht_matrix = compute_hht_matrix(signal, fs, matrix_size=256)
# plt.imshow(hht_matrix, aspect='auto', cmap='hot', origin='lower')
# plt.colorbar()
# plt.show()

## 8. Summary and Next Steps

This notebook provides:

1. **CNN Encoder Architecture**: 3-layer convolutional network with Instance Normalization and LeakyReLU
2. **SVM Classifier**: Multi-class classification using extracted CNN features
3. **End-to-End Model**: Optional fully trainable model with neural network classifier
4. **Data Generation**: Synthetic HHT matrix generation for demonstration
5. **HHT Computation**: Reference implementation for real sEMG signals

### For Real Data:
1. Load your sEMG signals
2. Apply HHT transformation using `compute_hht_matrix()`
3. Prepare labels for your classification task
4. Train the classifier using `sEMGHHTClassifier` or `sEMGHHTEndToEndClassifier`

### Hyperparameter Tuning:
- `base_channels`: Number of channels in first conv layer (default: 64)
- `svm_C`: SVM regularization parameter
- `svm_kernel`: SVM kernel type ('rbf', 'linear', 'poly')
- `learning_rate`: Learning rate for end-to-end training
- `dropout_rate`: Dropout rate in end-to-end model

In [ ]:
print("\n" + "="*60)
print("sEMG-HHT CNN Classifier - Ready for Use")
print("="*60)
print(f"\nDevice: {device}")
print(f"Encoder feature dimension: {encoder.get_feature_dim()}")
print(f"Number of classes: {len(class_names)}")
print(f"\nClass names:")
for i, name in enumerate(class_names):
    print(f"  {i}: {name}")